In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from helpers import (get_training_observations, 
                     get_training_labels)
import pandas as pd
import numpy as np
from joblib import dump
from joblib import load

# Model specific imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import metrics
tf.get_logger().setLevel('INFO')

import optuna

In [2]:
# Set up data
x_train = get_training_observations()
y_train = get_training_labels()
le = load('../model_joblibs/final_labelEncoder.joblib')
    
y_train_transformed = le.transform(y_train['EMPO_3'])
y_train_transformed

x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train_transformed, test_size=0.2, random_state=27)   

Getting all training observations from 'metagenome_classification.db'...
Getting all training labels from 'metagenome_classification.db'...


In [3]:
K = tf.keras.backend

def build_model(
    activation='relu',
    hidden_layer_sizes=[],
    final_layer_activation='softmax',
    dropout=0.0,
    optimizer='Adam',
    learning_rate=0.01,
    kernel_regularizer=1e-5,
    bias_regularizer=1e-5,
    activity_regularizer=1e-5,
    n_classes=15
):
    """Build a multi-class logistic regression model using Keras.

    Args:
      n_classes: Number of output classes in the dataset.
      hidden_layer_sizes: A list with the number of units in each hidden layer.
      activation: The activation function to use for the hidden layers.
      optimizer: The optimizer to use (SGD, Adam).
      learning_rate: The desired learning rate for the optimizer.

    Returns:
      model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten())
    
    for hidden_layer_size in hidden_layer_sizes:
        if hidden_layer_size > n_classes:
            model.add(tf.keras.layers.Dense(
                hidden_layer_size,
                activation=activation,
                kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
                bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
                activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
            ))
            if dropout > 0:
                model.add(tf.keras.layers.Dropout(dropout))

    model.add(tf.keras.layers.Dense(
        n_classes,
        activation=final_layer_activation,
        kernel_regularizer=tf.keras.regularizers.L2(kernel_regularizer),
        bias_regularizer=tf.keras.regularizers.L2(bias_regularizer),
        activity_regularizer=tf.keras.regularizers.L2(activity_regularizer)
    ))
    opt = None
    if optimizer == "SGD":
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
        raise f"Unsupported optimizer, {optimizer}"
    model.compile(
        loss="categorical_crossentropy", optimizer=opt, 
        metrics=[tfa.metrics.F1Score(num_classes=15, average='weighted')]
    )
    return model
    

In [4]:
def objective_3_layer(trial, x_tr=x_tr, x_val=x_val, y_tr=y_tr, y_val=y_val):
    
    # Set up XGBoost hyperparameter space
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01),
        'hidden_layer_size1': trial.suggest_int('hidden_layer_size1', 0, 1024),
        'hidden_layer_size2': trial.suggest_int('hidden_layer_size2', 0, 1024),
        'hidden_layer_size3': trial.suggest_int('hidden_layer_size3', 0, 1024),
        'kernel_regularizer': trial.suggest_float('kernel_regularizer', 1e-10, 1e-4),
        'bias_regularizer': trial.suggest_float('bias_regularizer', 1e-10, 1e-4),
        'activity_regularizer': trial.suggest_float('activity_regularizer', 1e-10, 1e-4),
        'dropout': trial.suggest_float('dropout', 0, 0.1)
    }
    
    hidden_layer_sizes = [params.pop('hidden_layer_size1'), params.pop('hidden_layer_size2'), params.pop('hidden_layer_size3')]
    
    y_tr_one_hot = K.one_hot(K.cast(y_tr, dtype='int64'), 15)
    y_val_one_hot = K.one_hot(K.cast(y_val, dtype='int64'), 15)
    
    # Create model
    nn = build_model(
        **params,
        n_classes=15,
        optimizer='Adam',
        final_layer_activation='softmax',
        hidden_layer_sizes=hidden_layer_sizes
    )
    
    # Fit model
    nn.fit(x=x_tr,
           y=y_tr_one_hot,
           epochs=50,
           batch_size=64,
           verbose=0)
    
    evaluation = nn.evaluate(x=x_val, y=y_val_one_hot, verbose=0, return_dict=True)
    f1_score = evaluation['f1_score']
    loss = evaluation['loss']
    
    print(f'F1 Score: {f1_score}; Loss={loss}')
    
    return f1_score
    

In [ ]:
study_nn_3 = optuna.create_study(direction='maximize')
study_nn_3.optimize(objective_3_layer, n_trials=200)

print('Number of finished trials:', len(study_nn_3.trials))
print('Best trial:', study_nn_3.best_trial.params)

[I 2022-08-03 01:34:41,897] A new study created in memory with name: no-name-5b8b4fd9-5075-4548-bbce-bdd291ccee1b


Metal device set to: Apple M1 Pro


2022-08-03 01:34:41.901291: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-03 01:34:41.901502: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-08-03 01:34:42.222862: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-03 01:34:43.185407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 01:35:16.862424: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 01:35:17,164] Trial 0 finished with value:

F1 Score: 0.9125401973724365; Loss=0.5355969071388245


2022-08-03 01:35:17.850159: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 01:35:36.694201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 01:35:37,036] Trial 1 finished with value: 0.8929336071014404 and parameters: {'learning_rate': 0.006754698625646383, 'hidden_layer_size1': 43, 'hidden_layer_size2': 889, 'hidden_layer_size3': 293, 'kernel_regularizer': 6.132842482914758e-05, 'bias_regularizer': 6.306411002393794e-05, 'activity_regularizer': 4.183009307221787e-05, 'dropout': 0.03914116316812516}. Best is trial 0 with value: 0.9125401973724365.


...

2022-08-03 01:55:19.850202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 01:55:49.035788: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 01:55:50,018] Trial 39 finished with value: 0.8918083906173706 and parameters: {'learning_rate': 0.008095809600484154, 'hidden_layer_size1': 736, 'hidden_layer_size2': 279, 'hidden_layer_size3': 415, 'kernel_regularizer': 9.3203645209411e-06, 'bias_regularizer': 5.7584366270286286e-05, 'activity_regularizer': 4.675275606315627e-05, 'dropout': 0.021084475466755986}. Best is trial 29 with value: 0.9176479578018188.


F1 Score: 0.8918083906173706; Loss=0.5156732201576233


2022-08-03 01:55:50.902566: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-03 01:56:17.248378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
[I 2022-08-03 01:56:18,142] Trial 40 finished with value: 0.8085407018661499 and parameters: {'learning_rate': 0.005000091264700382, 'hidden_layer_size1': 273, 'hidden_layer_size2': 15, 'hidden_layer_size3': 255, 'kernel_regularizer': 9.95388639758878e-05, 'bias_regularizer': 6.572750616012963e-05, 'activity_regularizer': 5.573566401006565e-05, 'dropout': 0.062114235008335045}. Best is trial 29 with value: 0.9176479578018188.


F1 Score: 0.8085407018661499; Loss=0.8262576460838318


2022-08-03 01:56:19.448851: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


* Trial 29 finished with value: 0.9176479578018188 and parameters: {'learning_rate': 0.006120059584156001, 'hidden_layer_size1': 6, 'hidden_layer_size2': 871, 'hidden_layer_size3': 424, 'kernel_regularizer': 7.04892717757341e-07, 'bias_regularizer': 8.077172989474054e-05, 'activity_regularizer': 4.125772221139609e-05, 'dropout': 0.03581861197995262}. Best is trial 29 with value: 0.9176479578018188.